In [28]:
import pandas as pd
import numpy as np
import glob

file_list = glob.glob('./mem_dumps/memory_dump_event_*.csv')

value_arrays = []

for filename in file_list:
    df = pd.read_csv(filename)
    vals = df['ch0'].to_numpy()
    vals_reshaped = vals.reshape(8, 8)   # Reshape to 8x8
    value_arrays.append(vals_reshaped)


In [32]:
value_arrays[2]

array([[ 0,  2,  0, -2,  1,  2,  0,  0],
       [ 0,  1,  0, -1,  0,  1,  0,  0],
       [ 0, -3,  1,  1,  0,  0,  0,  0],
       [ 0, -4,  0,  4,  0,  1, -1,  0],
       [ 0,  0,  0,  1,  0,  2, -2,  0],
       [ 0,  1,  0, -1,  0,  0,  2, -1],
       [ 0,  0,  0, -1,  0, -1, -1,  2],
       [ 0,  0,  0, -2,  0,  0, -1,  1]], dtype=int64)

[]